In [ ]:
# This converts input files of latitude and longitude, and calculates sunrise and sunset times

# Created May 2021 by E.S.

In [3]:
# There are two parts to this notebook:

# 1.) To take ArcGIS output csv files, extract relevant coordinates, and them write out to
#     another, 'processed' csv with additional info in a header
# 2.) To take processed csvs and calculate sunset and sunrise times for each coordinate

In [18]:
import ephem
import skyfield
import pandas as pd
import numpy as np

# PART 1

In [19]:
# read in raw file as written out by ArcGIS

df = pd.read_csv("./data/test.csv")

In [24]:
scotland_df = df.where(np.logical_and(df["Join_Count"] == 1,df["NAME_1"] == "Scotland")).dropna(how="all")

In [25]:
scotland_df

,OBJECTID,Join_Count,TARGET_FID,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,LAT,LON
3,6408.0,1.0,6408.0,GBR,United Kingdom,GBR.3_1,Scotland,Alba,,Home Nation|Constituent Country,Kingdom,,NaN,55.4907,-4.1026
4,6408.0,1.0,6408.0,GBR,United Kingdom,GBR.3_1,Scotland,Alba,,Home Nation|Constituent Country,Kingdom,,NaN,56.4907,-4.2026


In [34]:
# write out coordinates file with a header

# to add a more interesting header to the file, open a file object
processed_file_name = "test_processed.csv"
hdr = open(processed_file_name, "w")

hdr.write('KEY;VALUE;COMMENT;REF\n')
hdr.write('technical_string;scotland;;\n')
hdr.write('display_string;Scotland;;\n')
hdr.write('start_date;1707 May 1;Treaty of Union;Wikipedia\n')
hdr.write('end_date;-99999;;\n')

hdr.write('##############################\n')

scotland_df.to_csv(hdr)
print("Wrote out " + processed_file_name)

# PART 2

In [50]:
# Read processed csv back in

df_processed = pd.read_csv(processed_file_name,skiprows=6)
print("Reading in " + processed_file_name)

In [62]:
# https://rhodesmill.org/skyfield/almanac.html

from skyfield import api
from skyfield.api import Loader
from skyfield import almanac

load = Loader('~/skyfield-data')
ts = api.load.timescale()

# load JPL ephemeri
# de440.bsp: 31-DEC-1549 00:00 to   25-JAN-2650 00:00
# de441.bsp goes to 17191 AD
eph = api.load('de440.bsp') # change so that it doesn't have to be downloaded each time
#print(eph)

# extract start and end dates
t0 = ts.utc(1707, 5, 1, 4)
t1 = ts.utc(1707, 6, 2, 4)

for ind_num in range(0,len(df_processed)):
    bluffton = api.wgs84.latlon(df_processed["LAT"][ind_num], df_processed["LON"][ind_num])

    #t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(eph, bluffton))
    
    f = almanac.risings_and_settings(eph, eph['Sun'], bluffton)
    t, y = almanac.find_discrete(t0, t1, f)

    for ti, yi in zip(t, y):
        print(ti.utc_iso(), 'Rise' if yi else 'Set')

    #print(t.utc_iso())
    #print(y)

1707-05-01T04:37:35Z Rise
1707-05-01T19:49:17Z Set
1707-05-02T04:35:22Z Rise
1707-05-02T19:51:14Z Set
1707-05-03T04:33:11Z Rise
1707-05-03T19:53:10Z Set
1707-05-04T04:31:01Z Rise
1707-05-04T19:55:06Z Set
1707-05-05T04:28:53Z Rise
1707-05-05T19:57:02Z Set
1707-05-06T04:26:46Z Rise
1707-05-06T19:58:57Z Set
1707-05-07T04:24:40Z Rise
1707-05-07T20:00:52Z Set
1707-05-08T04:22:37Z Rise
1707-05-08T20:02:46Z Set
1707-05-09T04:20:35Z Rise
1707-05-09T20:04:40Z Set
1707-05-10T04:18:35Z Rise
1707-05-10T20:06:32Z Set
1707-05-11T04:16:36Z Rise
1707-05-11T20:08:25Z Set
1707-05-12T04:14:40Z Rise
1707-05-12T20:10:16Z Set
1707-05-13T04:12:45Z Rise
1707-05-13T20:12:06Z Set
1707-05-14T04:10:52Z Rise
1707-05-14T20:13:56Z Set
1707-05-15T04:09:02Z Rise
1707-05-15T20:15:45Z Set
1707-05-16T04:07:13Z Rise
1707-05-16T20:17:32Z Set
1707-05-17T04:05:27Z Rise
1707-05-17T20:19:19Z Set
1707-05-18T04:03:43Z Rise
1707-05-18T20:21:04Z Set
1707-05-19T04:02:01Z Rise
1707-05-19T20:22:48Z Set
1707-05-20T04:00:22Z Rise
1707-